<a href="https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab8/L8_Colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 8.1 Machine Learning 1 – Artificial Neural Network, Classification 

##Learning goals

Students will be able to:

1. Create code to plot time and frequency domain data for normal and abnormal data
2. Implement an artificial neural network to classify different datasets


##Introduction

In this lab, we will create a machine learning (ML) model based on the accelerometer (ADXL345) signals to predict the running conditions of the axial flow fan (AFF) which we used in Lab 2. An autoencoder we practiced in Prelab8 will be employed to determine whether AFF is in a normal or abnormal condition. Lab8 is broken down into two main sections: **1) Data collection**, and **2) Training ML model**. In the data collection section, we will collect accelerometer data when the machine is in normal and abnormal conditions using Raspberry Pi. The abnormal condition will be set of increasing eccentric force by adding a mass on a blade of the AFF as Lab2. In the training ML model section, we will utilize the given scripts in Prelab8 to train an autoencoder. In addition, we will perform feature engineering by doing signal processing to see the effects of input feature on the performance of the ML models. And the, we will save the selected and trained model to local disk to use it in the next lab. The schematic of Lab9 is illustrated in Figure 1.

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab8_fig1.png?raw=true" width="100%">

*Figure 1 Lab8: Schematic of anomaly detection model training for axial flow fan*

## Data Collection for Machine Learning

## Part 1: Data collection practice

First, make a wire connection between ADXL345 sensor and Raspberry Pi. If you are having trouble with the connection, please look at the instructions in Lab2 manual. After you make a connection, you are ready to collect data. Before attaching the sensor to the target placement, let’s try to get data and understand the output data format. 

The sample Python script ([Lab8_data_collector.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab8/Lab8_data_collector.py?raw=true)) below to collect ADXL345 accelerometer data is on Github repo. In the script, what you need to pay attention to is ‘condition_identifier’ and ‘duration’ variables in the middle of the script.

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Python.png?raw=tru" width="20">Python - Python3 ([Lab8_data_collector.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab8/Lab8_data_collector.py?raw=true))**

```
import time
import board
import busio
import adafruit_adxl34x
from micropython import const
import csv
import datetime

i2c = busio.I2C(board.SCL, board.SDA) # i2c variable defines I2C interfaces and GPIO pins using busio and board modules

acc = adafruit_adxl34x.ADXL345(i2c) # acc object is instantiation using i2c of Adafruit ADXL34X library

acc.data_rate = const(0b1111) # change sampling rate as 3200 Hz

# ratedict=output rate dictionary
# See Table5 of Lab2 manual key=rate code (decimal), value=output data rate (Hz)
ratedict = {15:3200,14:1600,13:800,12:400,11:200,10:100,9:50,8:25,7:12.5,6:6.25,5:3.13,4:1.56,3:0.78,2:0.39,1:0.2,0:0.1}

print("Output data rate is {} Hz".format(ratedict[acc.data_rate])) # printing out data rate

def getData(sensor:object, N:int): # sensor: ADXL sensor object, N: The number of sample in each timestamp.
    t1 = time.time()
    data_x = [] # initialize data_x to contain x-axis acceleration
    data_y = [] # initialize data_y to contain y-axis acceleration
    data_z = [] # initialize data_z to contain z-axis acceleration
    for i in range(N):
        x_acc, y_acc, z_acc = sensor.acceleration
        data_x.append(str(x_acc))
        data_y.append(str(y_acc))
        data_z.append(str(z_acc))
    x_data = ' '.join(data_x)
    y_data = ' '.join(data_y)
    z_data = ' '.join(data_z)
    return x_data, y_data, z_data # each data returns space delimited string each element is measurement of acceleration

condition_identifier = "Test" # condition identifier
duration = 10 # data collection duration in second unit

filename = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")+"_"+condition_identifier+"_lab8_data.csv"
header = ["Condition", "Xacc array [m/s2]", "Yacc array [m/s2]", "Zacc array [m/s2]"]
start = time.time()

print("== Data collection for {} measurements started. ==".format(duration))

with open(filename, 'w') as f: # Make and open file object
    write = csv.writer(f) # write object for the created file
    write.writerow(header) # write the first row (header)
    for j in range(duration): # for measurement durations
        x, y, z = getData(acc, 1000) # get x-, y-, z-axis acceleration array of 1000 data points (1 second) for each
        print('======= {}th of {} collection ======='.format(j+1, duration)) # Print out the progress 
        write.writerow([condition_identifier, x, y, z])
f.close()

print("== Data saving is done. == takes", time.time() - start)

```

---

Let’s try to collect data as a practice. Please set ‘condition_identifier’ as “test” and ‘duration’ as 10. If you run the script, your Raspberry Pi will collect accelerations for 10 seconds. According to Raspberry Pi specifications, the total time may be longer than the ‘duration’ you set. The output filename must be 
“YYYYMMDD_HHmmSS_Test_lab8_data.csv”. The date-time in the first part of the filename is the date-time when the script starts to be run. The “Test” in this case is the ‘conditon_identifier’. If you open the saved CSV file, you will see the collected data as Table 1. The first column (Condition column) indicates 
‘condition_identifier’. The second, third, and last columns indicate the measured accelerations for 1 second (1000 data points) for each axis, respectively. Because the sampling frequency is 1000Hz, each row means data for 1 second. Each array is space delimited float array. To practice data loading and transformation, perform Task 1. 

<br></br>
*Table 1 Example of collected data*

<table width="100%">
<thead>
  <tr>
    <th>Condition</th>
    <th>Xacc array [m/s2]</th>
    <th>Yacc array [m/s2]</th>
    <th>Zacc array [m/s2]</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td align="center" colspan="4">...</td>
  </tr>
  <tr>
    <td align="center">Test</td>
    <td align="center">x<sub>1</sub> x<sub>2</sub> … x<sub>999</sub> x<sub>1000</sub></td>
    <td align="center">y<sub>1</sub> y<sub>2</sub> … y<sub>999</sub> y<sub>1000</sub></td>
    <td align="center">z<sub>1</sub> z<sub>2</sub> … z<sub>999</sub> z<sub>1000</sub></td>
  </tr>
  <tr>
    <td align="center">Test</td>
    <td align="center">x<sub>1</sub> x<sub>2</sub> … x<sub>999</sub> x<sub>1000</sub></td>
    <td align="center">y<sub>1</sub> y<sub>2</sub> … y<sub>999</sub> y<sub>1000</sub></td>
    <td align="center">z<sub>1</sub> z<sub>2</sub> … z<sub>999</sub> z<sub>1000</sub></td>
  </tr>
  <tr>
    <td align="center">Test</td>
    <td align="center">x<sub>1</sub> x<sub>2</sub> … x<sub>999</sub> x<sub>1000</sub></td>
    <td align="center">y<sub>1</sub> y<sub>2</sub> … y<sub>999</sub> y<sub>1000</sub></td>
    <td align="center">z<sub>1</sub> z<sub>2</sub> … z<sub>999</sub> z<sub>1000</sub></td>
  </tr>
  <tr>
    <td align="center" colspan="4">...</td>
  </tr>
  <tr>
    <td align="center">Test</td>
    <td align="center">x<sub>1</sub> x<sub>2</sub> … x<sub>999</sub> x<sub>1000</sub></td>
    <td align="center">y<sub>1</sub> y<sub>2</sub> … y<sub>999</sub> y<sub>1000</sub></td>
    <td align="center">z<sub>1</sub> z<sub>2</sub> … z<sub>999</sub> z<sub>1000</sub></td>
  </tr>
  <tr>
    <td align="center">Test</td>
    <td align="center">x<sub>1</sub> x<sub>2</sub> … x<sub>999</sub> x<sub>1000</sub></td>
    <td align="center">y<sub>1</sub> y<sub>2</sub> … y<sub>999</sub> y<sub>1000</sub></td>
    <td align="center">z<sub>1</sub> z<sub>2</sub> … z<sub>999</sub> z<sub>1000</sub></td>
  </tr>
  <tr>
    <td align="center">Test</td>
    <td align="center">x<sub>1</sub> x<sub>2</sub> … x<sub>999</sub> x<sub>1000</sub></td>
    <td align="center">y<sub>1</sub> y<sub>2</sub> … y<sub>999</sub> y<sub>1000</sub></td>
    <td align="center">z<sub>1</sub> z<sub>2</sub> … z<sub>999</sub> z<sub>1000</sub></td>
  </tr>
  <tr>
    <td align="center" colspan="4">...</td>
  </tr>
</tbody>
</table>

### Task 1.1

After running ‘Lab8_data_collector.py’ with variables, ‘condition_identifier’=”Test” and ‘duration’=10, plot each axis data in both time-domain and frequency-domain as Figure 2. An example Python sciript for plotting data is given ([Lab8_plot_example.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab8/Lab8_plot_example.py?raw=true)). You can use the given Python script to plot data. However, it is strongly recommended to understand each line of the script.

* Capture the plot and attach it to the report below. 

  * Load the CSV file you collected in Python script. 

  * Add ‘condition_identifier’ and your name at the end of the title of each plot (e.g., ‘Time domain, Test, John Doe’). 

  * Your plots must include each axis label and units. 

  * Recall Lab2 and Prelab8. You have already done this work before.

  * Your data have 10 rows except the header row because you collected data for 10 seconds. Please select one of the data rows randomly. 
  * You can use either Raspberry Pi or laptop to plot the data. If you want, perform plotting the given code in the code block below this Colab Notebook.



<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab8_fig2.png?raw=true" width="100%">

*Figure 2 Data plot: Time domain (left) and Frequency domain (right)*




---

Place your screenshot for Task 1.1 here in case you use Raspberry Pi or Python on your laptop.

---


In [ ]:
# Perform Task 1.1 for plotting "Test" data here
# If you want to use this Colab Notebook, you have to load your file on
# 1) the Colab virtual directory
# 2) mounting your Google Colab drive
#




#

## Part 2: Data collection in normal condition

Before we deploy the sensor to the AFF, let’s check the hardware and the speed controller. The hardware configuration and the speed controller are shown in Figure 3. For safety reasons, the base part is fixed on the table using tapes. Do not remove the tapes because it maybe move when you run the AFF due to aerodynamic forces. To turn on the fan, rotate the knob of the speed controller clockwise. The relationship between knob pointer placements and the actual rotational speed of AFF is shown in Table 2. When you rotate the pointer of the knob to L (Speed 1 in Figure 3 (right)), for example, the AFF rotates around 1800 rpm. By adjusting the control knob, you can increase and decrease the rotating speed of the fan. Different from the vacuum pump case of Prelab9, it is obvious that changing rotational speed makes an ML model development hard. On the other hand, it may be more interesting. For instance, if you attach the unbalanced mass to a blade of the fan at the lowest rotational speed, does the amplitude of vibration be bigger than the maximum rotational speed without any attached mass? We don’t know the answer exactly but probably no. Turn the fan on and adjust the knob to see if the rotation speed changes well. 

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab8_fig3.png?raw=true" width="100%">

*Figure 3 Configuration of axial flow fan (left) and speed controller (right)*
<br></br>
*Table 2 Controller indicator vs. rotational speed of AFF*

<table width="60%">
<thead>
  <tr>
    <th align="center">Speed indiator</th>
    <th align="center">Rotational Speed</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td align="center">1 (L, Low)</td>
    <td align="center">1800 rpm</td>
  </tr>
  <tr>
    <td align="center">2</td>
    <td align="center">2150 rpm</td>
  </tr>
  <tr>
    <td align="center">3 (M, Middle)</td>
    <td align="center">2500 rpm</td>
  </tr>
  <tr>
    <td align="center">4</td>
    <td align="center">2750 rpm</td>
  </tr>
  <tr>
    <td align="center">5 (H, High)</td>
    <td align="center">3000 rpm</td>
  </tr>
</tbody>
</table>

Deploy the accelerometer (ADXL345) on the top of the fan as Figure 4. **You have to remember the sensor placement and the axis configuration for the next lab again.** If you are not sure how to set up the accelerometer to the AFF, please look at [Lab2.2 manual](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab2/L2_Colab2.ipynb). To collect acceleration data when the machine is in normal conditions for 5 minutes, perform Task 1.2. While you collect the data, try to change the rotational speed. For example, while collecting, set Speed 1 (1800 rpm) for the first 1 minute, and then Speed 2 (2150 rpm) between 1 and 2 minutes and so forth. The example of the experimental table is shown in Table 3. You see the progress of your data collector in the Shell of Thonny or Terminal window of Raspberry Pi as Figure 5. For this, please perform data collection at once with other members sharing the table and the AFF. 

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab8_fig4.png?raw=true" width="50%">

*Figure 4 Sensor placement on top of the AFF*
<br></br>
*Table 3 Experimental table for data collection of AFF*

<table width="60%">
<thead>
  <tr>
    <th align="center">Speed indiator</th>
    <th align="center">Rotational Speed</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td align="center">1 (L, Low)</td>
    <td align="center">1800 rpm</td>
  </tr>
  <tr>
    <td align="center">2</td>
    <td align="center">2150 rpm</td>
  </tr>
  <tr>
    <td align="center">3 (M, Middle)</td>
    <td align="center">2500 rpm</td>
  </tr>
  <tr>
    <td align="center">4</td>
    <td align="center">2750 rpm</td>
  </tr>
  <tr>
    <td align="center">5 (H, High)</td>
    <td align="center">3000 rpm</td>
  </tr>
</tbody>
</table>

<br></br>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab8_fig5.png?raw=true" width="80%">

*Figure 5 Data collection progress check: Thonny (left) and Terminal (right)*

### Task 1.2

Run ‘Lab9_data_collector.py’ while you run the AFF with variables, ‘condition_identifier’=”Normal” and ‘duration’=300, plot each axis data in both time-domain and frequency-domain as Figure 2. 

* Capture the plots according to the rotational speed and attach these to the report below. 

  *	Load the CSV file in Python script. 

  *	Add ‘condition_identifier’, rotational speed, and your name at the end of the title of each plot (e.g., ‘Time domain, Normal, 1800 rpm, John Doe’). 

  *	Your plots must include each axis label and units. 

  *	**Plot two different rotational speeds and compare them** For example, one is 1800 rpm and another is 2500 rpm. 

* Your data have 300 rows except the header row because you collected data for 300 seconds. Note that the actual collection time is probably loger than 300 seconds because of data saving time. Please select one of the data rows randomly within one-speed range. For example, if you plot Speed 2, you need to select a data row between 61 and 120. 






---

Place your screenshot for Task 1.2 (speed condition 1) here in case you use Raspberry Pi or Python on your laptop.

---


---

Place your screenshot for Task 1.2 (speed condition 2) here in case you use Raspberry Pi or Python on your laptop.

---

In [ ]:
# Perform Task 1.2 for plotting "Test" data here
# If you want to use this Colab Notebook, you have to load your file on
# 1) the Colab virtual directory
# 2) mounting your Google Colab drive
#




#

## Part 3: Data collection in abnormal condition

Next step is to collect abnormal condition data by adding mass on the blade for the ML model. There will be an adhesive putty on top of the AFF. Using the putty, make the fan unbalanced as Figure 6. If you are not sure how to set up the abnormal condition by adding mass, please review [Lab2.2 manual](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab2/L2_Colab2.ipynb). Please safely disassemble and assemble the AFF. If you are unsure about your setup, please ask TA. After setup for the abnormal condition of the AFF, perform TASK 3.

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab8_fig6.png?raw=true" width="50%">

*Figure 6 Unbalanced fan blade for abnormal condition of AFF*

### Task 1.3 (Repeat Task 1.2 in *abnormal* condition)

Run ‘Lab8_data_collector.py’ while you run the AFF with variables, ‘condition_identifier’=”Abnormal” and ‘duration’=300, plot each axis data in both time-domain and frequency-domain as Figure 2. 

* Capture the plots according to the rotational speed and attach these to the report below. 

  *	Load the CSV file in Python script. 

  *	Add ‘condition_identifier’, rotational speed, and your name at the end of the title of each plot (e.g., ‘Time domain, Abnormal, 1800 rpm, John Doe’). 

  *	Your plots must include each axis label and units. 

  *	**Plot two different rotational speeds and compare them** For example, one is 1800 rpm and another is 2500 rpm. 

* Your data have 300 rows except the header row because you collected data for 300 seconds. Note that the actual collection time is probably loger than 300 seconds because of data saving time. Please select one of the data rows randomly within one-speed range. For example, if you plot Speed 2, you need to select a data row between 61 and 120. 

---

Place your screenshot for Task 1.3 (speed condition 1) here in case you use Raspberry Pi or Python on your laptop.

---


---

Place your screenshot for Task 1.3 (speed condition 2) here in case you use Raspberry Pi or Python on your laptop.

---

In [ ]:
# Perform Task 1.3 for plotting "Test" data here
# If you want to use this Colab Notebook, you have to load your file on
# 1) the Colab virtual directory
# 2) mounting your Google Colab drive
#




#

### Task 1.4

Upload the CSV files for Task 1.1-1.3 on Brightspace alongside your lab submission.

  *	You do not need to upload the Python script. 

  *	Leave the CSV filenames as generated. 


<br></br>

Please continue to [Prelab 8.2 here](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab8/L8_Colab2.ipynb).
